<a href="https://colab.research.google.com/github/nameshigawa/AE_AnsysBBoxViewer/blob/main/python/ansys_video_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ansys Video Analyzer (Google Colab)

This notebook analyzes a video using YOLO and exports bounding box data as JSON.
The output JSON can be visualized in After Effects using AE_AnsysBBoxViewer.

### 1. Install Required Libraries

First, we need to install the `opencv-python` library for video processing and `ultralytics` which provides the YOLO model for object detection. Run the cell below to install them.

In [ ]:
!pip install opencv-python ultralytics pyyaml

### 2. Run Script

Run the script, select an MP4 file, and a JSON file containing the analysis results will be downloaded automatically.

In [ ]:
import cv2
import json
import os
import yaml
import urllib.request
from ultralytics import YOLO
from google.colab import files

# ========================================
# Load configuration from GitHub
# ========================================
REPO_URL = "https://raw.githubusercontent.com/nameshigawa/AE_AnsysBBoxViewer/main/python"
CONFIG_FILE = "custom_bytetrack.yaml"

print("Downloading configuration...")
config_url = f"{REPO_URL}/{CONFIG_FILE}"

try:
    with urllib.request.urlopen(config_url) as response:
        config_content = response.read().decode('utf-8')
    
    CONFIG = yaml.safe_load(config_content)
    print("✓ Configuration loaded successfully!")
    print("\nConfiguration:")
    for key, value in CONFIG.items():
        if isinstance(value, dict):
            print(f"  {key}:")
            for k, v in value.items():
                print(f"    {k}: {v}")
        else:
            print(f"  {key}: {value}")
except Exception as e:
    print(f"Warning: Could not load config from GitHub: {e}")
    print("Using default configuration...")
    CONFIG = {
        'tracker_type': 'bytetrack',
        'track_high_thresh': 0.6,
        'track_low_thresh': 0.1,
        'new_track_thresh': 0.7,
        'track_buffer': 60,
        'match_thresh': 0.85,
        'fuse_score': True,
        'motion': {'max_iou_distance': 0.7}
    }

# ========================================
# Upload and analyze video
# ========================================
print("\nUploading video...")
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

cap = cv2.VideoCapture(video_path)

model = YOLO("yolo11m.pt")

results_json = []
frame_count = 0

print("Analyzing video...")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)
    frame_boxes = []

    for r in results:
        for i, box in enumerate(r.boxes):
            b = box.xyxy[0].tolist()
            conf = float(box.conf[0])
            cls = int(box.cls[0])

            frame_boxes.append({
                "id": i,
                "x": int(b[0]),
                "y": int(b[1]),
                "width": int(b[2] - b[0]),
                "height": int(b[3] - b[1]),
                "label": model.names[cls],
                "conf": round(conf, 3)
            })

    results_json.append(frame_boxes)
    frame_count += 1

cap.release()

print(f"✓ Analysis complete! ({frame_count} frames processed)")

# ========================================
# Save and download JSON
# ========================================
base, _ = os.path.splitext(video_path)
json_path = base + "_bbox.json"

with open(json_path, "w") as f:
    json.dump(results_json, f, indent=2)

print(f"✓ JSON saved: {json_path}")
print("Downloading file...")
files.download(json_path)